<a href="https://colab.research.google.com/github/SufyAD/TS-Projects/blob/master/Copy_of_Customer_Support_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture --no-stderr
%pip install -qU langgraph langchain-community langchain_google_genai langchain_core langgraph-checkpoint-sqlite

In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


# _set_env("GEMINI_API_KEY")
# _set_env("LANGCHAIN_API_KEY")

### Populate database (Optional)
Populating our database with some dummydata to perform different operations such as `get_order_status`, `cancel_order`, `update_order`

In [ ]:
!mkdir -p state_db && [ ! -f state_db/example.db ] && wget -P state_db https://github.com/langchain-ai/langchain-academy/raw/main/module-2/state_db/example.db

--2024-12-25 10:06:46--  https://github.com/langchain-ai/langchain-academy/raw/main/module-2/state_db/example.db
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/langchain-ai/langchain-academy/main/module-2/state_db/example.db [following]
--2024-12-25 10:06:46--  https://raw.githubusercontent.com/langchain-ai/langchain-academy/main/module-2/state_db/example.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 110592 (108K) [application/octet-stream]
Saving to: ‘state_db/example.db’

example.db          100%[===================>] 108.00K  --.-KB/s    in 0.02s   

2024-12-25 10:06:46 (4.43 MB/s) - ‘s

In [ ]:
# -------------------- Dummy Order ----------------------

from langchain.tools import tool
from typing_extensions import TypedDict
order_json: TypedDict = {
  "orders": [
      {
          "order_id": "12345",
          "status": "Processing",
          "expected_delivery": "2024-12-23",
          "items": [
              {"name": "Wireless Headphones", "category": "Audio", "price": 15000, "quantity": 1},
              {"name": "Smart Watch", "category": "Wearables", "price": 12000, "quantity": 1}
          ]
      },
      {
          "order_id": "67890",
          "status": "Shipped",
          "expected_delivery": "2024-12-25",
          "items": [
              {"name": "Gaming Mouse", "category": "Accessories", "price": 7000, "quantity": 1},
              {"name": "Mechanical Keyboard", "category": "Accessories", "price": 10000, "quantity": 1}
          ]
      },
      {
          "order_id": "11223",
          "status": "Delivered",
          "expected_delivery": "2024-12-20",
          "items": [
              {"name": "Portable Speaker", "category": "Audio", "price": 8000, "quantity": 1},
              {"name": "Smartphone Stand", "category": "Accessories", "price": 2000, "quantity": 2},
              {"name": "USB-C Hub", "category": "Connectivity", "price": 3000, "quantity": 1}
          ]
      }
  ]
}

In [ ]:
#----------------------- Products in DB -----------------------
products = [
    {"name": "Wireless Headphones", "category": "Audio", "price": 15000, "stock": 10},
    {"name": "Smart Watch", "category": "Wearables", "price": 12000, "stock": 100},
    {"name": "Gaming Mouse", "category": "Accessories", "price": 5000, "stock": 50},
    {"name": "Mechanical Keyboard", "category": "Accessories", "price": 8000, "stock": 30},
    {"name": "Portable Power Bank", "category": "Energy", "price": 4000, "stock": 200},
    {"name": "Bluetooth Speaker", "category": "Audio", "price": 6000, "stock": 40},
    {"name": "Smart Glasses", "category": "Wearables", "price": 18000, "stock": 15},
    {"name": "Fitness Tracker", "category": "Wearables", "price": 7000, "stock": 80},
    {"name": "Drone with Camera", "category": "Drones", "price": 25000, "stock": 10},
    {"name": "4K Action Camera", "category": "Photography", "price": 15000, "stock": 25},
    {"name": "VR Headset", "category": "Gaming", "price": 22000, "stock": 20},
]

In [ ]:
import sqlite3
db_path = "state_db/example.db"
conn = sqlite3.connect(db_path, check_same_thread=False) #conn is basically enables us to connect with our SQLite3 database otherwise we could not setup
cursor = conn.cursor()

In [ ]:
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS products (
        name TEXT PRIMARY KEY,
        category TEXT,
        price INTEGER,
        stock INTEGER
    )
    """
)
for product in products:
    cursor.execute("""
    INSERT OR REPLACE INTO products (name, category, price, stock)
    VALUES (?, ?, ?, ?)
    """,(product["name"].lower(), product["category"], product["price"], product["stock"]))

    conn.commit()

In [ ]:
cursor.execute("SELECT * FROM products")
rows = cursor.fetchall()
for row in rows:
    print(row)

('wireless headphones', 'Audio', 15000, 10)
('smart watch', 'Wearables', 12000, 100)
('gaming mouse', 'Accessories', 5000, 50)
('mechanical keyboard', 'Accessories', 8000, 30)
('portable power bank', 'Energy', 4000, 200)
('bluetooth speaker', 'Audio', 6000, 40)
('smart glasses', 'Wearables', 18000, 15)
('fitness tracker', 'Wearables', 7000, 80)
('drone with camera', 'Drones', 25000, 10)
('4k action camera', 'Photography', 15000, 25)
('vr headset', 'Gaming', 22000, 20)


In [ ]:
from langgraph.checkpoint.sqlite import SqliteSaver
memory: SqliteSaver = SqliteSaver(conn)

In [ ]:
# #Cleaing DB
cursor = conn.cursor()
# cursor.execute("DELETE FROM order_items")
# cursor.execute("DELETE FROM orders")

# # Optionally, you can reset the autoincrement if needed:
# # cursor.execute("DELETE FROM sqlite_sequence WHERE name='orders'")
# # cursor.execute("DELETE FROM sqlite_sequence WHERE name='order_items'")

# # Commit the changes and close the connection
# conn.commit()
# cursor.close()
# conn.close()

# print("Database cleaned and ready for new data.")

In [ ]:
#creating tables if doesn't exist
cursor.execute("""
    CREATE TABLE IF NOT EXISTS orders (
        order_id TEXT PRIMARY KEY,
        status TEXT,
        expected_delivery TEXT
    )
    """)

cursor.execute("""
    CREATE TABLE IF NOT EXISTS order_items (
        order_id TEXT,
        name TEXT,
        category TEXT,
        price INTEGER,
        quantity INTEGER,
        FOREIGN KEY (order_id) REFERENCES orders(order_id)
    )
""")

# Populating data in the database
for order in order_json["orders"]:
    cursor.execute("""
    INSERT OR REPLACE INTO orders (order_id, status, expected_delivery)
    VALUES (?, ?, ?)
    """, (order["order_id"], order["status"], order["expected_delivery"]))

    # Now iterate over the items for each order and insert them into the order_items table
    for item in order["items"]:
        cursor.execute("""
        INSERT OR REPLACE INTO order_items (order_id, name, category, price, quantity)
        VALUES (?, ?, ?, ?, ?)
        """, (order["order_id"], item["name"].lower(), item["category"], item["price"], item["quantity"]))

conn.commit()

In [ ]:
cursor.execute("""
  SELECT * FROM orders
""")
orders = cursor.fetchall()

cursor.execute("SELECT * FROM order_items")
order_items = cursor.fetchall()

In [ ]:
# Print the data
print("Orders:")
for order in orders:
    print(order)

print("\nOrder Items:")
for item in order_items:
    print(item)

Orders:
('12345', 'Processing', '2024-12-23')
('67890', 'Shipped', '2024-12-25')
('11223', 'Delivered', '2024-12-20')
('823', 'Processing', '2024-12-31')

Order Items:
('12345', 'Wireless Headphones', 'Audio', 15000, 1)
('12345', 'Smart Watch', 'Wearables', 12000, 1)
('67890', 'Gaming Mouse', 'Accessories', 7000, 1)
('67890', 'Mechanical Keyboard', 'Accessories', 10000, 1)
('11223', 'Portable Speaker', 'Audio', 8000, 1)
('11223', 'Smartphone Stand', 'Accessories', 2000, 2)
('11223', 'USB-C Hub', 'Connectivity', 3000, 1)
('823', 'gaming mouse', 'Accessories', 5000, 1)


In [ ]:
conn.commit()
cursor.close()

**Tools**
Next, define our assistant's `tools`

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from IPython.display import Image, display

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.graph.state import CompiledStateGraph

from langchain_core.messages import HumanMessage, SystemMessage

from typing_extensions import TypedDict

In [ ]:
from google.colab import userdata
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=userdata.get("GEMINI_API_KEY"))
# llm_with_tools = llm.bind_tools(tools) # to be added

In [ ]:
llm.invoke("Hello")

AIMessage(content='Hello there! How can I help you today?\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-d5776f5b-c6ef-4790-b62b-4d02da10db5a-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [ ]:
from datetime import datetime, timedelta
from pydantic import BaseModel
from typing import Optional

class OrderState(TypedDict): #to handle order udpates
    order_id: str
    status: str
    expected_delivery: str

class State(MessagesState):
    orders_json: dict[str, list[OrderState]]

In [ ]:
from time import time
from random import randint

def get_user_input() -> dict:
    while True:
        try:
            product_name = input("Enter the product name: ").strip().lower()  # Ensuring lowercase for matching
            quantity = int(input("Enter the quantity: ").strip())

            if quantity <= 0:
                raise ValueError("Quantity must be a positive integer.")

            return {"product_name": product_name, "quantity": quantity}
        except ValueError as e:
            print(f"Error: {e}")


# Function to match product from the database
def match_product(db_path: str, product_name: str, quantity: int) -> dict:
    query = """
        SELECT name, category, price, stock
        FROM products
        WHERE name = ?
    """
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute(query, (product_name,))
        product = cursor.fetchone()

        if not product:
            raise ValueError(f"Product '{product_name}' not found in the database.")

        name, category, price, stock = product

        if quantity > stock:
            raise ValueError(f"Insufficient stock for '{name}'. Available: {stock}")

        return {
            "name": name,
            "category": category,
            "price": price,
            "quantity": quantity
        }

In [ ]:
# Function to create an order
def create_order(db_path: str, matched_product: dict) -> dict:

    order_id = randint(100, 999)
    expected_delivery = "2024-12-31"  # Example date logic

    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()

        # Insert into orders table
        query = """
        INSERT INTO orders (order_id, status, expected_delivery)
        VALUES (?, ?, ?)
        """
        cursor.execute(query, (order_id, "Processing", expected_delivery)) #for every new order we'll pass status as PROCESSING

        # Insert into order_items table
        query_item = """
        INSERT INTO order_items (order_id, name, category, price, quantity)
        VALUES (?, ?, ?, ?, ?)
        """
        cursor.execute(query_item, (
            order_id,
            matched_product["name"],
            matched_product["category"],
            matched_product["price"],
            matched_product["quantity"]
        ))

        # Update stock in products table
        query_stock_update = """
            UPDATE products SET stock = stock - ? WHERE name = ?
        """
        cursor.execute(query_stock_update, (matched_product["quantity"], matched_product["name"]))
        conn.commit()

    return {
        "order_id": order_id,
        "status": "Processing",
        "expected_delivery": expected_delivery,
        "items": [matched_product],
    }

In [ ]:
@tool
def process_order(db_path: str) -> dict:
    """Handles product order creation, checking stock, and updating the database."""
    try:
        # Get user input for product and quantity
        user_input = get_user_input()

        # Match product from the database
        matched_product = match_product(db_path, user_input["product_name"], user_input["quantity"])

        # Create the order in the database
        order_details = create_order(db_path, matched_product)

        return order_details
    except ValueError as e:
        return {"error": str(e)}

In [ ]:
result = process_order("state_db/example.db")
print(result)

Enter the product name: gaming mouse
Enter the quantity: 1
{'order_id': 823, 'status': 'Processing', 'expected_delivery': '2024-12-31', 'items': [{'name': 'gaming mouse', 'category': 'Accessories', 'price': 5000, 'quantity': 1}]}


In [ ]:
# try:
#   user_input = get_user_input()
#   matched_product = match_product_in_database(user_input["product_name"], user_input["quantity"])
#   order = create_order(user_input, matched_product)
#   print(f"Order successfully created: {order}")
# except Exception as e:
#   print(f"Error: {e}")

Error: name 'get_user_input' is not defined


In [ ]:
from langchain_core.runnables import RunnableConfig

@tool
def get_order_status(config: RunnableConfig) -> list[dict]:
    """
    Retrieves the status of an order based on its unique order ID.

    Args:
        config (RunnableConfig): Contains configuration for the order ID.

    Returns:
        List[Dict]: A list of dictionaries with order details, including:
            - `order_id` (str): The unique ID of the order.
            - `status` (str): The current status of the order (e.g., "Processing", "Shipped").
            - `expected_delivery` (str): The expected delivery date of the order (optional).
            - `items` (list of dict): The items related to the order.
    """
    # Get order_id from the config
    configuration = config.get("configurable", {})
    order_id = configuration.get("order_id", None)

    if not order_id:
        raise ValueError("order_id is required")

    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Update SQL query to correctly join orders and order_items tables
    query = """
    SELECT
        orders.order_id,
        orders.status,
        orders.expected_delivery,
        order_items.name,
        order_items.category,
        order_items.price,
        order_items.quantity
    FROM
        orders
    JOIN
        order_items ON orders.order_id = order_items.order_id
    WHERE
        orders.order_id = ?
    """
    cursor.execute(query, (order_id,))
    rows = cursor.fetchall()

    # Create a list of dictionaries for the rows retrieved from the database
    column_names = [column[0] for column in cursor.description]
    results = [dict(zip(column_names, row)) for row in rows]

    cursor.close()
    conn.close()

    if not results:
        raise ValueError(f"Order with ID {order_id} not found")

    return results

In [ ]:
# config = RunnableConfig(configurable={"order_id": "12345"})

# # Call the function to fetch order status
# results = get_order_status(config)

# # Print the result
# print(results)


### Cancelling an order
Right now it is kept simple that a common json of orders is passed through the functions and keep updating the content of this json, later we will replace this json with our database

In [ ]:
import sqlite3
from langchain_core.runnables import RunnableConfig

@tool
def cancel_order(config: RunnableConfig) -> str:
    """
    Cancels the order based on its unique order ID.

    Args:
        config (RunnableConfig): The config containing the order ID.

    Returns:
        str: A message indicating whether the order was successfully canceled or not.
    """
    configuration = config.get("configurable", {})
    order_id = configuration.get("order_id", None)

    if not order_id:
        raise ValueError("order_id is required")

    # Connect to SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Check if the order exists and get its status
    cursor.execute("""
        SELECT order_id, status FROM orders WHERE order_id = ?
    """, (order_id,))
    order = cursor.fetchone()

    if not order:
        cursor.close()
        conn.close()
        return f"Order with ID {order_id} not found."

    # If the order status is not "Delivered", we cancel it
    if order[1] != "Delivered":
        cursor.execute("""
            DELETE FROM orders WHERE order_id = ?
        """, (order_id,))
        cursor.execute("""
            DELETE FROM order_items WHERE order_id = ?
        """, (order_id,))
        conn.commit()
        cursor.close()
        conn.close()
        return f"Order {order_id} has been successfully canceled."

    cursor.close()
    conn.close()
    return f"Order {order_id} cannot be canceled as it has already been {order[1]}."


In [ ]:
# config = RunnableConfig(configurable={"order_id": "67890"})
# cancel_result = cancel_order("67890", config)
# print(cancel_result)

### Update an Order
Modifying the common json which will be replaced with our DB later


In [ ]:
# from datetime import datetime, timedelta
# updated_order = {
#     "status": "Updated",
#     "expected_delivery": datetime.now() + timedelta(days=3),  # current date + 3-4 days
#     "items": [
#         {"name": "Smartphone Stand", "category": "Accessories", "price": 2000, "quantity": 2},
#     ]
# }

In [ ]:
@tool
def update_order(order_id: str, config: RunnableConfig)->OrderState:
    """
    Updates an order by its unique order ID with new details provided by an LLM.

    Args:
        order_id (str): The unique identifier of the order to update.
        order_json (dict): A dictionary containing the list of orders.
        state (State): Application state, including LLM input and messages.

    Returns:
        dict: A dictionary containing:
            - `message` (str): Confirmation or error message.
            - `updated_order` (dict): Details of the updated order (if applicable).
    """

    # updated_order = llm.invoke(state['messages'][-1])

    for i, order in enumerate(order_json["orders"]):
        if order["order_id"] == order_id:
            if order["status"] != "Delivered":
                order["status"] = updated_order["status"]
                order["expected_delivery"] = (datetime.now() + timedelta(days=3)).strftime("%Y-%m-%d")
                order["items"] = updated_order["items"]

                return {
                    "message": f"Order {order_id} has been successfully updated.",
                    "updated_order": order
                }
            else:
                return {
                    "message": f"Order {order_id} has been delivered and cannot be updated.",
                    "status": order["status"]
                }

In [ ]:
from langchain_core import tools
tools = [get_order_status, cancel_order]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
llm_with_tools.invoke("Hi")

In [ ]:
from google.colab import userdata

def assistant(state: MessagesState):
  """
  Handles the processing of conversational messages using a language model with tools.
  """
  sys_msg = SystemMessage(content="""You are a helpful Order Management Assistant,
  you have to be always humble and positive with the user's query even if the customer loses his tone;
  Keep the situation under control""")

  return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}


In [ ]:
agent: StateGraph = StateGraph(MessagesState)

agent.add_node("assistant", assistant)
agent.add_node("tools", ToolNode(tools))

agent.add_edge(START, "assistant")
agent.add_conditional_edges(
    "assistant",
    tools_condition,
)

agent.add_edge("tools", "assistant")

agent.set_entry_point("assistant")

### Adding Checkpointer right now - later we will replace this with out DB


In [ ]:
memory: MemorySaver = MemorySaver()
graph: CompiledStateGraph = agent.compile(checkpointer=memory)

In [ ]:
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# Input
initial_input = {"messages": "Yes Please cancel it"}

# Thread
thread = {"configurable": {
    "thread_id": "3",
    "order_id" : "12345"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

In [ ]:
# Example usage
try:
    updated_order = update_order("67890", order_json, updated_order)
    print(updated_order)
except ValueError as e:
    print(e)